In [1]:
"""
Calculate fire severity statistics within AFD observations from MODIS and VIIRS.
Severity index: Composite Burn Severity (CBI), calculated in GEE via Parks (2018)

Author: maxwell.cook@colorado.edu
"""

import os, sys
import ee
import geemap

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

print("Success !")

Success !


In [3]:
# Load the gridded FRP data
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/AFD/viirs_snpp_jpss1_afd_gridstats')
print(f"Number of grid cells: {grid.size().getInfo()}")
grid = grid.select(['grid_index','Fire_ID'])
print(grid.first().propertyNames().getInfo())

Number of grid cells: 56974
['system:index', 'grid_index', 'Fire_ID']


In [6]:
# folder containing the individual CBI rasters for each fire
cbidir = 'projects/jfsp-aspen/assets/CBI_V2/'
# list out all the images
cbi_images = ee.data.listAssets({'parent': cbidir})['assets']
cbi_image_list = [
    ee.Image(f['id']) for f in cbi_images if f['id'].endswith('CBI_bc')
]
print(f"There are [{len(cbi_image_list)}] CBI images")
print(cbi_image_list[0].projection().getInfo())
print(grid.first().geometry().projection().getInfo())

There are [76] CBI images
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [0.00026949458523585647, 0, -108.03337236183873, 0, -0.00026949458523585647, 37.627103485215514]}
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


In [7]:
# create a 'forest mask' from the TreeMap data
treemap = ee.ImageCollection("USFS/GTAC/TreeMap/v2016")
print(f"TreeMap bands available for analysis:\n\n{treemap.first().bandNames().getInfo()}")

TreeMap bands available for analysis:

['ALSTK', 'BALIVE', 'CANOPYPCT', 'CARBON_D', 'CARBON_DWN', 'CARBON_L', 'DRYBIO_D', 'DRYBIO_L', 'FLDSZCD', 'FLDTYPCD', 'FORTYPCD', 'GSSTK', 'QMD_RMRS', 'SDIPCT_RMRS', 'STANDHT', 'STDSZCD', 'TPA_DEAD', 'TPA_LIVE', 'Value', 'VOLBFNET_L', 'VOLCFNET_D', 'VOLCFNET_L']


In [10]:
# create a forest mask and check on it
forest = treemap.select('FORTYPCD').mosaic().gt(0)
# grab a CBI image and update the mask to check
test = ee.ImageCollection.fromImages(cbi_image_list)

# mask the CBI images
def mask_forest(img):
    return img.updateMask(forest)
test_masked = test.map(mask_forest)

# Initialize a map
Map = geemap.Map()
Map.addLayerControl()

Map.centerObject(grid, zoom=8)

Map.addLayer(forest)
Map.addLayer(test)
Map.addLayer(test_masked)
Map.addLayer(grid)

Map

Map(center=[38.60112970313928, -106.09596143712085], controls=(WidgetControl(options=['position', 'transparent…

In [11]:
# Setup seperate functions for average/stdev and extremes (percentiles)
def calculate_average(cbi_image, grid_fc, fire_id):
    """Calculate mean and standard deviation of CBI within each grid cell."""
    # mask the CBI to forest pixels
    cbi_image = cbi_image.updateMask(forest)
    # perform the reduction
    return cbi_image.reduceRegions(
        collection=grid_fc,
        reducer=ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(), sharedInputs=True
        ),
        scale=30
    ).map(lambda f: f.set("Fire_ID", fire_id))

def calculate_extremes(cbi_image, grid_fc, fire_id):
    """Calculate percentiles (90th, 95th, 99th) of CBI within each grid cell."""
    # mask the CBI to forest pixels
    cbi_image = cbi_image.updateMask(forest)
    # perform the reduction
    return cbi_image.reduceRegions(
        collection=grid_fc,
        reducer=ee.Reducer.percentile([90, 95, 97, 99]),
        scale=30
    ).map(lambda f: f.set("Fire_ID", fire_id))

def merge_collections(primary, secondary, join_field="grid_index"):
    """Join two FeatureCollections and merge their properties."""
    join = ee.Join.inner()
    filter_condition = ee.Filter.equals(
        leftField=join_field, 
        rightField=join_field
    )
    joined = join.apply(primary, secondary, filter_condition)

    # Flatten joined results
    def merge_features(f):
        primary_props = ee.Feature(f.get("primary")).toDictionary()
        secondary_props = ee.Feature(f.get("secondary")).toDictionary()
        combined_props = primary_props.combine(secondary_props)
        return ee.Feature(None, combined_props)

    return joined.map(merge_features)

print("Functions ready !")

Functions ready !


In [13]:
print(cbi_image_list[2].get('fireID').getInfo())

103


In [14]:
cbi_image_list[0]

In [15]:
# Initialize export tasks for all fires
# Initialize an empty collection to hold all results
all_fires_stats = ee.FeatureCollection([])
for cbi_image in tqdm(cbi_image_list, desc="Processing Fires"):
    
    # Extract Fire ID from the image file name
    fire_id = cbi_image.get('fireID').getInfo()
    # Filter grid cells for this fire
    fire_grid = grid.filter(ee.Filter.eq("Fire_ID", fire_id))

    # Calculate mean/stdDev and percentiles
    average_stats = calculate_average(cbi_image, fire_grid, fire_id)
    extremes_stats = calculate_extremes(cbi_image, fire_grid, fire_id)

    # Merge results
    stats = merge_collections(average_stats, extremes_stats)

    # Add to the global collection
    all_fires_stats = all_fires_stats.merge(stats)

# export table to Drive.
export_task = ee.batch.Export.table.toDrive(
    collection=all_fires_stats,
    description='gridstats_cbibc_forest_v2',
    fileNamePrefix='gridstats_cbibc_forest_v2',
    fileFormat='CSV', 
    folder='CBI'
)

export_task.start()
print("Export to Earth Engine Asset started!")
monitor_export(export_task, 360) # 360=every 5 min

Processing Fires:   0%|          | 0/76 [00:00<?, ?it/s]

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Export completed successfully !!!!
